# Encapsulamiento
¿Que es el **encapsulamiento**? ¿Con que se come? Y sobre todo... ¿Por que es relevante?

Pues bien, el encapsulamiento es uno de los 4 principio fundamental de la POO: abstraccion, herencia, polimorfismo y **encapsulamiento**. Basicamente nos ayuda a esconder atributos e implementaciones internas de las clases que no queremos que sean de acceso publico.

¿A que me refiero con acceso **publico**?

Resulta que, por defecto en Python todos los metodos (funciones) y atributos de una clase son de acceso publico, esto quiere decir que otras clases pueden acceder a los mismos sin restricciones.  

Esto resulta problematico cuando mi clase tiene metodos o atributos a los cuales otros no deberian de poder acceder, o dado el caso, si deberian de poder acceder pero solo a traves de metodos concretos para controlar el acceso. 

A estos metodos especiales de acceso los llamamos **getters** y **setters**.


## Acceso publico

In [1]:
class PublicPerson:
    def __init__(self, name: str, age: int, income: int):
        self.name = name
        self.age = age
        self.income = income


class PublicDog:
    def __init__(self, owner: PublicPerson, name: str, age: int):
        self.owner = owner
        self.name = name
        self.age = age

    def bark(self) -> str:
        return f"{self.name} says: my owner's income is {self.owner.income}!"


class PublicBankLoan:
    MEDIUM_INCOME: float = range(3500, 7000)

    def __init__(self, principal: PublicPerson, amount: float, duration: int):
        self.principal = principal
        self.amount = amount
        self.duration = duration

    def calculate_interest(self) -> float:
        if self.principal.income < self.MEDIUM_INCOME.start:
            return 0.07
        elif self.principal.income in self.MEDIUM_INCOME:
            return 0.05
        return 0.03  # high income

In [2]:
shaggy: PublicPerson = PublicPerson("Shaggy", 30, 6000)
loan = PublicBankLoan(shaggy, 10000, 12)

In [3]:
loan.calculate_interest()

0.05

Ahora, el acceso publico es complejo ya fuera de ambas clases y solo por medio de una instancid de la clase PublicBankLoan yo podria acceder a la persona y cambiar sus ingresos reportados, lo cual no tiene sentido, desde fuera yo no deberia de poder acceder ni cambiar los ingresos reportados. 

In [4]:
loan.principal.income = 200
loan.principal.income

200

In [5]:
loan.calculate_interest()  # la tasa de interes es mas alta ya que cambie el income a ingresos bajos

0.07

Esto es una falla de seguridad enorme, ya que desde una clase que no tiene nada que ver yo podria acceder a estos atributos.

In [6]:
scooby_doo: PublicDog = PublicDog(shaggy, "Scooby Doo", 5)
scooby_doo.bark()

"Scooby Doo says: my owner's income is 200!"

Scooby Doo ladra y nos dice el income de su dueño, para Scooby Doo es normal hablar, pero no deberia de saber los ingresos de su dueño.

Con este ejemplo pueden los peligros de dejar todos los atributos de una clase como publicos.

## Acceso privado
Ahora veamos como podemos restringir el acceso y edicion a los atributos y metodos de una clase

In [ ]:
class PrivatePerson:
    CANNOT_SET_INCOME_MSG: str = "Cannot set income directly"

    def __init__(self, name: str, age: int, income: int):
        self.name = name  # no hay problema si el nombre es publico
        self.age = age
        self.__income = income  # asi privatizamos un atributo

    @property
    def income(self) -> int:  # getter, obtenemos el valor de __income
        return self.__income

    @income.setter
    def income(self, _: int):
        raise AttributeError(
            self.CANNOT_SET_INCOME_MSG
        )  # setter, no se puede modificar el valor de __income


class PrivateDog:
    def __init__(self, owner: PrivatePerson, name: str, age: int):
        self.owner = owner
        self.name = name
        self.age = age

    def bark(self) -> str:
        return f"{self.name} says: my owner's income is {self.owner.income}!"


class PrivateBankLoan:
    MEDIUM_INCOME: float = range(3500, 7000)

    def __init__(self, principal: PrivatePerson, amount: float, duration: int):
        self.principal = principal
        self.amount = amount
        self.duration = duration

    def calculate_interest(self) -> float:
        if self.principal.income < self.MEDIUM_INCOME.start:
            return 0.07
        elif self.principal.income in self.MEDIUM_INCOME:
            return 0.05
        return 0.03  # high income

In [8]:
fred: PrivatePerson = PrivatePerson("Fred", 30, 6000)
scooby_doo: PrivateDog = PrivateDog(fred, "Scooby Doo", 5)

Ahora intentemos que Scooby nos diga el income de Fred

In [9]:
scooby_doo.bark()

"Scooby Doo says: my owner's income is 6000!"

Tambien veamos si desde la clase de prestamo bancario se puede editar los ingresos de Shaggy.

In [10]:
fred_loan = PrivateBankLoan(fred, 10000, 12)
fred_loan.principal.income = 200

AttributeError: Cannot set income directly

In [11]:
fred.income  # podemos obtener el valor de income, pero no podemos modificarlo directamente

6000

Ahora, como podemos dejar que ciertas clases si modifiquen el income de las personas? Tal vez queremos que una aplicacion del banco tenga acceso a poder hacer esto.

In [ ]:
class BankApplication:
    def __init__(self, principal: PrivatePerson):
        self.__principal = principal

    def change_income(self, new_income: int):
        self.__principal.income = new_income  # Cambiamos el income a traves de un setter privado


class PrivatePerson:
    CANNOT_SET_INCOME_MSG: str = "Cannot set income directly"

    def __init__(self, name: str, age: int, income: int):
        self.name = name  # no hay problema si el nombre es publico
        self.age = age
        self.__income = income  # asi privatizamos un atributo

    def __get_caller_class(self) -> type:  # asi se privatiza un metodo
        """Obtiene la clase que está intentando acceder al setter"""
        import inspect  # noqa: PLC0415

        frame = inspect.currentframe()
        try:
            # Subimos dos frames para obtener la clase que llama
            caller_frame = frame.f_back.f_back
            if "self" in caller_frame.f_locals:
                return caller_frame.f_locals["self"].__class__
        finally:
            del frame
        return None

    @property
    def income(self) -> int:  # getter, obtenemos el valor de __income
        return self.__income

    @income.setter
    def income(self, new_income: int) -> None:
        caller_class = self.__get_caller_class()
        if isinstance(caller_class, BankApplication):
            self.__income = new_income
            return
        raise AttributeError(
            self.CANNOT_SET_INCOME_MSG
        )  # setter, no se puede modificar el valor de __income

In [13]:
daphne: PrivatePerson = PrivatePerson("Daphne", 30, 10000)
application: BankApplication = BankApplication(daphne)

In [14]:
daphne.income

10000

In [15]:
scooby_doo: PrivateDog = PrivateDog(daphne, "Scooby Doo", 5)
scooby_doo.owner.income = 200

AttributeError: Cannot set income directly